
=== Entraînement CNN ===



Chargement FP: 100%|██████████| 1926/1926 [00:19<00:00, 97.35it/s] 


Epoch 1/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.5816 - auc: 0.4921 - loss: 0.0820 - precision: 0.6003 - recall: 0.9110
Epoch 1: val_auc improved from -inf to 0.51242, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 17s 148ms/step - accuracy: 0.5818 - auc: 0.4922 - loss: 0.0819 - precision: 0.6002 - recall: 0.9116 - val_accuracy: 0.5977 - val_auc: 0.5124 - val_loss: 0.0906 - val_precision: 0.6009 - val_recall: 0.9665 - learning_rate: 3.0000e-04
Epoch 2/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6106 - auc: 0.5539 - loss: 0.0698 - precision: 0.6087 - recall: 0.9893
Epoch 2: val_auc improved from 0.51242 to 0.53917, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6105 - auc: 0.5540 - loss: 0.0698 - precision: 0.6085 - recall: 0.9892 - val_accuracy: 0.6082 - val_auc: 0.5392 - val_loss: 0.0829 - val_precision: 0.6032 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 3/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6190 - auc: 0.5786 - loss: 0.0681 - precision: 0.6139 - recall: 0.9894
Epoch 3: val_auc improved from 0.53917 to 0.56467, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6189 - auc: 0.5786 - loss: 0.0681 - precision: 0.6138 - recall: 0.9895 - val_accuracy: 0.6071 - val_auc: 0.5647 - val_loss: 0.0765 - val_precision: 0.6026 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6307 - auc: 0.5867 - loss: 0.0674 - precision: 0.6211 - recall: 0.9911
Epoch 4: val_auc improved from 0.56467 to 0.57570, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.6307 - auc: 0.5867 - loss: 0.0674 - precision: 0.6211 - recall: 0.9911 - val_accuracy: 0.6124 - val_auc: 0.5757 - val_loss: 0.0725 - val_precision: 0.6058 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 5/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6399 - auc: 0.6078 - loss: 0.0656 - precision: 0.6263 - recall: 0.9963
Epoch 5: val_auc improved from 0.57570 to 0.63826, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6398 - auc: 0.6078 - loss: 0.0656 - precision: 0.6261 - recall: 0.9962 - val_accuracy: 0.6261 - val_auc: 0.6383 - val_loss: 0.0700 - val_precision: 0.6143 - val_recall: 1.0000 - learning_rate: 3.0000e-04
Epoch 6/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6417 - auc: 0.5995 - loss: 0.0656 - precision: 0.6272 - recall: 0.9976
Epoch 6: val_auc improved from 0.63826 to 0.64403, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6415 - auc: 0.5994 - loss: 0.0656 - precision: 0.6270 - recall: 0.9975 - val_accuracy: 0.6376 - val_auc: 0.6440 - val_loss: 0.0681 - val_precision: 0.6220 - val_recall: 0.9982 - learning_rate: 3.0000e-04
Epoch 7/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6472 - auc: 0.6154 - loss: 0.0650 - precision: 0.6315 - recall: 0.9942
Epoch 7: val_auc improved from 0.64403 to 0.64597, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6470 - auc: 0.6149 - loss: 0.0650 - precision: 0.6313 - recall: 0.9942 - val_accuracy: 0.6397 - val_auc: 0.6460 - val_loss: 0.0658 - val_precision: 0.6239 - val_recall: 0.9947 - learning_rate: 3.0000e-04
Epoch 8/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6456 - auc: 0.6186 - loss: 0.0645 - precision: 0.6304 - recall: 0.9941
Epoch 8: val_auc did not improve from 0.64597
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6455 - auc: 0.6188 - loss: 0.0645 - precision: 0.6302 - recall: 0.9941 - val_accuracy: 0.6418 - val_auc: 0.6378 - val_loss: 0.0634 - val_precision: 0.6247 - val_recall: 0.9982 - learning_rate: 3.0000e-04
Epoch 9/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6497 - auc: 0.6261 - loss: 0.0637 - precision: 0.6328 - recall: 0.9960
Epoch 9: val_auc improved from 0.64597 to 0.66249, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6496 - auc: 0.6262 - loss: 0.0637 - precision: 0.6326 - recall: 0.9959 - val_accuracy: 0.6481 - val_auc: 0.6625 - val_loss: 0.0628 - val_precision: 0.6295 - val_recall: 0.9947 - learning_rate: 3.0000e-04
Epoch 10/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6511 - auc: 0.6283 - loss: 0.0636 - precision: 0.6338 - recall: 0.9957
Epoch 10: val_auc did not improve from 0.66249
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6509 - auc: 0.6286 - loss: 0.0636 - precision: 0.6336 - recall: 0.9956 - val_accuracy: 0.6439 - val_auc: 0.6544 - val_loss: 0.0635 - val_precision: 0.6264 - val_recall: 0.9965 - learning_rate: 3.0000e-04
Epoch 11/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6553 - auc: 0.6417 - loss: 0.0627 - precision: 0.6363 - recall: 0.9974
Epoch 11: val_auc improved from 0.66249 to 0.66434, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6552 - auc: 0.6419 - loss: 0.0627 - precision: 0.6362 - recall: 0.9974 - val_accuracy: 0.6544 - val_auc: 0.6643 - val_loss: 0.0620 - val_precision: 0.6343 - val_recall: 0.9912 - learning_rate: 3.0000e-04
Epoch 12/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6578 - auc: 0.6339 - loss: 0.0629 - precision: 0.6388 - recall: 0.9930
Epoch 12: val_auc did not improve from 0.66434
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6577 - auc: 0.6337 - loss: 0.0629 - precision: 0.6386 - recall: 0.9930 - val_accuracy: 0.6439 - val_auc: 0.6565 - val_loss: 0.0621 - val_precision: 0.6267 - val_recall: 0.9947 - learning_rate: 3.0000e-04
Epoch 13/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6597 - auc: 0.6689 - loss: 0.0611 - precision: 0.6396 - recall: 0.9955
Epoch 13: val_auc did not improve from 0.66434
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6596 - auc: 0.6684 - loss: 0.0611 - precision: 0.6395 - reca

60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6590 - auc: 0.6398 - loss: 0.0627 - precision: 0.6399 - recall: 0.9910 - val_accuracy: 0.6450 - val_auc: 0.6654 - val_loss: 0.0620 - val_precision: 0.6271 - val_recall: 0.9965 - learning_rate: 3.0000e-04
Epoch 15/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6682 - auc: 0.6703 - loss: 0.0608 - precision: 0.6452 - recall: 0.9967
Epoch 15: val_auc did not improve from 0.66545
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6679 - auc: 0.6701 - loss: 0.0608 - precision: 0.6449 - recall: 0.9967 - val_accuracy: 0.6450 - val_auc: 0.6585 - val_loss: 0.0618 - val_precision: 0.6271 - val_recall: 0.9965 - learning_rate: 3.0000e-04
Epoch 16/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6633 - auc: 0.6617 - loss: 0.0611 - precision: 0.6419 - recall: 0.9967
Epoch 16: val_auc did not improve from 0.66545
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6632 - auc: 0.6615 - loss: 0.0611 - precision: 0.6417 - reca

60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6634 - auc: 0.6625 - loss: 0.0608 - precision: 0.6422 - recall: 0.9943 - val_accuracy: 0.6544 - val_auc: 0.6656 - val_loss: 0.0613 - val_precision: 0.6340 - val_recall: 0.9929 - learning_rate: 3.0000e-04
Epoch 19/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6660 - auc: 0.6571 - loss: 0.0613 - precision: 0.6448 - recall: 0.9909
Epoch 19: val_auc improved from 0.66562 to 0.66750, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6659 - auc: 0.6573 - loss: 0.0613 - precision: 0.6446 - recall: 0.9910 - val_accuracy: 0.6555 - val_auc: 0.6675 - val_loss: 0.0612 - val_precision: 0.6347 - val_recall: 0.9929 - learning_rate: 3.0000e-04
Epoch 20/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6725 - auc: 0.6676 - loss: 0.0603 - precision: 0.6485 - recall: 0.9953
Epoch 20: val_auc did not improve from 0.66750
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6723 - auc: 0.6676 - loss: 0.0603 - precision: 0.6482 - recall: 0.9953 - val_accuracy: 0.6492 - val_auc: 0.6620 - val_loss: 0.0610 - val_precision: 0.6299 - val_recall: 0.9965 - learning_rate: 3.0000e-04
Epoch 21/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6715 - auc: 0.6722 - loss: 0.0600 - precision: 0.6474 - recall: 0.9973
Epoch 21: val_auc did not improve from 0.66750
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6714 - auc: 0.6722 - loss: 0.0600 - precision: 0.6472 - reca

60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6726 - auc: 0.6735 - loss: 0.0601 - precision: 0.6493 - recall: 0.9909 - val_accuracy: 0.6607 - val_auc: 0.6694 - val_loss: 0.0613 - val_precision: 0.6396 - val_recall: 0.9859 - learning_rate: 3.0000e-04
Epoch 24/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6726 - auc: 0.6859 - loss: 0.0598 - precision: 0.6488 - recall: 0.9936
Epoch 24: val_auc did not improve from 0.66941
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6724 - auc: 0.6860 - loss: 0.0598 - precision: 0.6486 - recall: 0.9936 - val_accuracy: 0.6555 - val_auc: 0.6618 - val_loss: 0.0609 - val_precision: 0.6347 - val_recall: 0.9929 - learning_rate: 3.0000e-04
Epoch 25/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6762 - auc: 0.6892 - loss: 0.0590 - precision: 0.6516 - recall: 0.9929
Epoch 25: val_auc improved from 0.66941 to 0.67140, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6761 - auc: 0.6896 - loss: 0.0590 - precision: 0.6514 - recall: 0.9929 - val_accuracy: 0.6649 - val_auc: 0.6714 - val_loss: 0.0603 - val_precision: 0.6409 - val_recall: 0.9947 - learning_rate: 3.0000e-04
Epoch 26/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6759 - auc: 0.6780 - loss: 0.0598 - precision: 0.6515 - recall: 0.9926
Epoch 26: val_auc did not improve from 0.67140
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6758 - auc: 0.6782 - loss: 0.0598 - precision: 0.6513 - recall: 0.9926 - val_accuracy: 0.6555 - val_auc: 0.6695 - val_loss: 0.0606 - val_precision: 0.6344 - val_recall: 0.9947 - learning_rate: 3.0000e-04
Epoch 27/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6801 - auc: 0.6894 - loss: 0.0591 - precision: 0.6542 - recall: 0.9935
Epoch 27: val_auc did not improve from 0.67140
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6800 - auc: 0.6891 - loss: 0.0591 - precision: 0.6540 - reca

60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6827 - auc: 0.6967 - loss: 0.0582 - precision: 0.6557 - recall: 0.9953 - val_accuracy: 0.6597 - val_auc: 0.6813 - val_loss: 0.0598 - val_precision: 0.6370 - val_recall: 0.9965 - learning_rate: 3.0000e-04
Epoch 32/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6840 - auc: 0.7064 - loss: 0.0576 - precision: 0.6569 - recall: 0.9940
Epoch 32: val_auc did not improve from 0.68132
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6838 - auc: 0.7063 - loss: 0.0576 - precision: 0.6567 - recall: 0.9940 - val_accuracy: 0.6597 - val_auc: 0.6722 - val_loss: 0.0603 - val_precision: 0.6382 - val_recall: 0.9894 - learning_rate: 3.0000e-04
Epoch 33/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6846 - auc: 0.6979 - loss: 0.0576 - precision: 0.6570 - recall: 0.9959
Epoch 33: val_auc did not improve from 0.68132
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6845 - auc: 0.6982 - loss: 0.0576 - precision: 0.6568 - reca

60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.6977 - auc: 0.7499 - loss: 0.0550 - precision: 0.6676 - recall: 0.9907 - val_accuracy: 0.6828 - val_auc: 0.6838 - val_loss: 0.0605 - val_precision: 0.6561 - val_recall: 0.9824 - learning_rate: 1.5000e-04
Epoch 45/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6917 - auc: 0.7320 - loss: 0.0558 - precision: 0.6630 - recall: 0.9919
Epoch 45: val_auc improved from 0.68375 to 0.68475, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.6916 - auc: 0.7320 - loss: 0.0559 - precision: 0.6629 - recall: 0.9920 - val_accuracy: 0.6733 - val_auc: 0.6847 - val_loss: 0.0598 - val_precision: 0.6481 - val_recall: 0.9877 - learning_rate: 1.5000e-04
Epoch 46/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6980 - auc: 0.7473 - loss: 0.0550 - precision: 0.6671 - recall: 0.9942
Epoch 46: val_auc did not improve from 0.68475
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6979 - auc: 0.7474 - loss: 0.0550 - precision: 0.6670 - recall: 0.9942 - val_accuracy: 0.6723 - val_auc: 0.6818 - val_loss: 0.0591 - val_precision: 0.6467 - val_recall: 0.9912 - learning_rate: 1.5000e-04
Epoch 47/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6977 - auc: 0.7451 - loss: 0.0548 - precision: 0.6672 - recall: 0.9930
Epoch 47: val_auc did not improve from 0.68475
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.6976 - auc: 0.7452 - loss: 0.0548 - precision: 0.6670 - reca

60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.7078 - auc: 0.7645 - loss: 0.0532 - precision: 0.6740 - recall: 0.9961 - val_accuracy: 0.6765 - val_auc: 0.6859 - val_loss: 0.0590 - val_precision: 0.6504 - val_recall: 0.9877 - learning_rate: 7.5000e-05
Epoch 56/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6974 - auc: 0.7640 - loss: 0.0538 - precision: 0.6675 - recall: 0.9906
Epoch 56: val_auc improved from 0.68591 to 0.69358, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.6973 - auc: 0.7640 - loss: 0.0538 - precision: 0.6673 - recall: 0.9907 - val_accuracy: 0.6796 - val_auc: 0.6936 - val_loss: 0.0587 - val_precision: 0.6527 - val_recall: 0.9877 - learning_rate: 7.5000e-05
Epoch 57/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.6992 - auc: 0.7721 - loss: 0.0530 - precision: 0.6690 - recall: 0.9900
Epoch 57: val_auc improved from 0.69358 to 0.69414, saving model to /kaggle/working/best_cnn_exoplanet.h5


60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.6991 - auc: 0.7722 - loss: 0.0530 - precision: 0.6688 - recall: 0.9901 - val_accuracy: 0.6786 - val_auc: 0.6941 - val_loss: 0.0589 - val_precision: 0.6516 - val_recall: 0.9894 - learning_rate: 7.5000e-05
Epoch 58/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7055 - auc: 0.7685 - loss: 0.0530 - precision: 0.6735 - recall: 0.9908
Epoch 58: val_auc did not improve from 0.69414
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.7054 - auc: 0.7686 - loss: 0.0530 - precision: 0.6733 - recall: 0.9909 - val_accuracy: 0.6775 - val_auc: 0.6887 - val_loss: 0.0594 - val_precision: 0.6515 - val_recall: 0.9859 - learning_rate: 7.5000e-05
Epoch 59/100
59/60 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.7078 - auc: 0.7785 - loss: 0.0528 - precision: 0.6749 - recall: 0.9928
Epoch 59: val_auc did not improve from 0.69414
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.7076 - auc: 0.7782 - loss: 0.0528 - precision: 0.6746 - reca


=== Classification Report ===
              precision    recall  f1-score   support

          FP     0.8482    0.2468    0.3823       385
          CP     0.6548    0.9700    0.7818       567

    accuracy                         0.6775       952
   macro avg     0.7515    0.6084    0.5820       952
weighted avg     0.7330    0.6775    0.6202       952


✅ Modèle et seuil sauvegardés dans /kaggle/working/

[VISUALISATION] Génération des graphiques...


✔ Graphiques sauvegardés dans /kaggle/working/cnn_results.png


In [7]:
# ==============================================================
# Détection d'Exoplanètes - Multi-Architecture (Kaggle)
# ==============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from pathlib import Path
from astropy.io import fits
from tqdm import tqdm
import warnings
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

warnings.filterwarnings('ignore')

# =============================
# CONFIGURATION
# =============================
tf.random.set_seed(42)
np.random.seed(42)

# -----------------------------
# PARAMÈTRES MODIFIABLES
# -----------------------------
model_type = 'cnn'  # Choisir: 'resnet', 'lstm', 'tcn', 'transformer', 'cnn'
data_path = '/kaggle/input/datasetfit/dataset_fits'  # Dataset Kaggle
target_length = 3197
output_path = '/kaggle/working/'  # Tout sera sauvegardé ici

# ------------------------------------------------------------------
# FONCTIONS DE CHARGEMENT ET NORMALISATION
# ------------------------------------------------------------------
def load_fits_file(filepath):
    try:
        with fits.open(filepath) as hdul:
            for ext in range(len(hdul)):
                data = hdul[ext].data
                if data is not None:
                    flux_columns = ['PDCSAP_FLUX', 'SAP_FLUX', 'FLUX', 'flux']
                    for col in flux_columns:
                        if hasattr(data, col):
                            flux = data[col]
                            return np.nan_to_num(flux, nan=0.0, posinf=0.0, neginf=0.0)
                    if isinstance(data, np.ndarray):
                        if data.ndim == 1:
                            return np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
                        elif data.ndim == 2:
                            flux = data[:, 0] if data.shape[1] > 0 else data[0, :]
                            return np.nan_to_num(flux, nan=0.0, posinf=0.0, neginf=0.0)
        print(f"⚠️  Aucune donnée de flux trouvée dans {filepath}")
        return None
    except Exception as e:
        print(f"Erreur lors du chargement de {filepath}: {e}")
        return None

def normalize_length(flux, target_length):
    current_length = len(flux)
    if current_length == target_length:
        return flux
    elif current_length > target_length:
        indices = np.linspace(0, current_length - 1, target_length, dtype=int)
        return flux[indices]
    else:
        old_indices = np.arange(current_length)
        new_indices = np.linspace(0, current_length - 1, target_length)
        return np.interp(new_indices, old_indices, flux)

def robust_normalize(flux):
    median = np.median(flux)
    mad = np.median(np.abs(flux - median))
    if mad == 0: mad = 1e-6
    return (flux - median) / mad

def load_fits_dataset(base_path, split='train', target_length=3197, test_size=0.2):
    from sklearn.model_selection import train_test_split
    base_path = Path(base_path)
    cp_path = base_path / 'CP'
    fp_path = base_path / 'FP'
    if not cp_path.exists() or not fp_path.exists():
        raise ValueError("Les dossiers CP/ et FP/ sont introuvables")
    X, y = [], []
    for f in tqdm(list(cp_path.glob("*.fits")), desc="Chargement CP"):
        flux = load_fits_file(f)
        if flux is not None and len(flux) > 50:
            X.append(normalize_length(flux, target_length))
            y.append(1)
    for f in tqdm(list(fp_path.glob("*.fits")), desc="Chargement FP"):
        flux = load_fits_file(f)
        if flux is not None and len(flux) > 50:
            X.append(normalize_length(flux, target_length))
            y.append(0)
    X = np.array(X)
    y = np.array(y)
    if len(X) == 0:
        raise ValueError("Aucune donnée FITS valide chargée")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=42
    )
    if split=='train': return X_train, y_train
    elif split=='test': return X_test, y_test
    else: raise ValueError("split doit être 'train' ou 'test'")

# =============================
# ARCHITECTURES
# =============================
def build_cnn(input_shape):
    inputs = keras.Input(shape=input_shape)

    x = layers.Conv1D(32, kernel_size=7, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(64, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.Conv1D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)

    x = layers.GlobalAveragePooling1D()(x)

    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    return models.Model(inputs, outputs)


def residual_block(x, filters, kernel_size=3, stride=1, name=''):
    shortcut = x
    x = layers.Conv1D(filters, kernel_size, strides=stride, padding='same',
                      kernel_regularizer=keras.regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv1D(filters, kernel_size, padding='same',
                      kernel_regularizer=keras.regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, 1, strides=stride, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def build_resnet(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv1D(32, 7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(3, strides=2, padding='same')(x)
    x = residual_block(x, 32)
    x = residual_block(x, 32)
    x = residual_block(x, 64, stride=2)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

def build_lstm(input_shape):
    inputs = keras.Input(shape=input_shape)
    x = layers.LSTM(128, return_sequences=True)(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LSTM(64, return_sequences=True)(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LSTM(32, return_sequences=False)(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

def build_tcn(input_shape, num_filters=64, kernel_size=3, dilations=[1,2,4,8,16]):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for i, dilation in enumerate(dilations):
        conv = layers.Conv1D(num_filters, kernel_size, padding='causal',
                             dilation_rate=dilation, activation='relu')(x)
        conv = layers.BatchNormalization()(conv)
        conv = layers.Dropout(0.3)(conv)
        if x.shape[-1] != num_filters:
            x = layers.Conv1D(num_filters, 1, padding='same')(x)
        x = layers.Add()([x, conv])
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

def build_transformer(input_shape, num_heads=4, ff_dim=128, num_blocks=3):
    inputs = keras.Input(shape=input_shape)
    x = layers.Dense(64)(inputs)
    for i in range(num_blocks):
        attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=64//num_heads)(x, x)
        attn_output = layers.Dropout(0.3)(attn_output)
        x = layers.Add()([x, attn_output])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        ffn = layers.Dense(ff_dim, activation='relu')(x)
        ffn = layers.Dropout(0.3)(ffn)
        ffn = layers.Dense(64)(ffn)
        x = layers.Add()([x, ffn])
        x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

# =============================
# FOCAL LOSS
# =============================
def focal_loss(gamma=2.0, alpha=0.25):
    def fl(y_true, y_pred):
        epsilon = keras.backend.epsilon()
        y_pred = keras.backend.clip(y_pred, epsilon, 1.0 - epsilon)
        y_true = tf.cast(y_true, tf.float32)
        ce = -y_true * keras.backend.log(y_pred)
        weight = alpha * y_true * keras.backend.pow(1 - y_pred, gamma)
        ce_neg = -(1 - y_true) * keras.backend.log(1 - y_pred)
        weight_neg = (1 - alpha) * (1 - y_true) * keras.backend.pow(y_pred, gamma)
        loss = weight * ce + weight_neg * ce_neg
        return keras.backend.mean(loss)
    return fl

# =============================
# ENTRAÎNEMENT
# =============================
def train_model(model_type='lstm', data_path=data_path, target_length=target_length):
    print(f"\n=== Entraînement {model_type.upper()} ===\n")
    X_train, y_train = load_fits_dataset(data_path, split='train', target_length=target_length)
    X_test, y_test = load_fits_dataset(data_path, split='test', target_length=target_length)
    
    # Normalisation robuste
    X_train = np.array([robust_normalize(f) for f in X_train])
    X_test = np.array([robust_normalize(f) for f in X_test])
    X_train = np.clip(X_train, -10, 10).reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = np.clip(X_test, -10, 10).reshape(X_test.shape[0], X_test.shape[1], 1)

    input_shape = (X_train.shape[1], 1)
    if model_type.lower()=='resnet': model = build_resnet(input_shape)
    elif model_type.lower()=='lstm': model = build_lstm(input_shape)
    elif model_type.lower()=='tcn': model = build_tcn(input_shape)
    elif model_type.lower()=='transformer': model = build_transformer(input_shape)
    elif model_type.lower()== 'cnn' : model = build_cnn(input_shape)
    else: raise ValueError("Modèle non reconnu")

    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

    model.compile(optimizer=keras.optimizers.Adam(3e-4),
                  loss=focal_loss(alpha=0.75),
                  metrics=[keras.metrics.BinaryAccuracy(name='accuracy'),
                           keras.metrics.Precision(name='precision'),
                           keras.metrics.Recall(name='recall'),
                           keras.metrics.AUC(name='auc')])

    callbacks = [
        EarlyStopping(monitor='val_auc', patience=20, restore_best_weights=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-7, verbose=1),
        ModelCheckpoint(f'{output_path}best_{model_type}_exoplanet.h5', monitor='val_auc',
                        mode='max', save_best_only=True, verbose=1)
    ]

    history = model.fit(X_train, y_train,
                        epochs=100,
                        batch_size=64,
                        validation_data=(X_test, y_test),
                        class_weight=class_weight_dict,
                        callbacks=callbacks,
                        verbose=1)

    # Évaluation
    y_pred_proba = model.predict(X_test).flatten()
    thresholds = np.arange(0.55,0.66,0.01)
    best_f1, best_threshold = 0, 0.6
    for th in thresholds:
        y_pred_temp = (y_pred_proba>th).astype(int)
        prec = np.sum((y_pred_temp==1)&(y_test==1))/max(np.sum(y_pred_temp==1),1)
        rec = np.sum((y_pred_temp==1)&(y_test==1))/max(np.sum(y_test==1),1)
        f1 = 2*prec*rec/max(prec+rec,1e-6)
        if f1>best_f1: best_f1, best_threshold = f1, th
    y_pred = (y_pred_proba>best_threshold).astype(int)
    print("\n=== Classification Report ===")
    print(classification_report(y_test, y_pred, target_names=['FP','CP'], digits=4, zero_division=0))

    # Sauvegarde
    model.save(f'{output_path}{model_type}_exoplanet_fits_final.h5')
    with open(f'{output_path}{model_type}_best_threshold_fits.pkl','wb') as f: pickle.dump(best_threshold,f)
    print(f"\n✅ Modèle et seuil sauvegardés dans {output_path}")

    # =============================
    # VISUALISATIONS
    # =============================
    print("\n[VISUALISATION] Génération des graphiques...")
    
    cm = confusion_matrix(y_test, y_pred)
    
    plt.style.use('seaborn-v0_8-darkgrid')
    fig = plt.figure(figsize=(16, 10))
    
    # 1. Loss
    plt.subplot(2, 3, 1)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    plt.title(f'{model_type.upper()} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # 2. Accuracy
    plt.subplot(2, 3, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # 3. AUC
    plt.subplot(2, 3, 3)
    plt.plot(history.history['auc'], label='Train AUC', linewidth=2)
    plt.plot(history.history['val_auc'], label='Val AUC', linewidth=2)
    plt.title('AUC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # 4. Matrice de confusion
    plt.subplot(2, 3, 4)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['FP', 'CP'],
                yticklabels=['FP', 'CP'])
    plt.title('Matrice de Confusion')
    plt.xlabel('Classe prédite')
    plt.ylabel('Classe réelle')
    
    # 5. Courbe ROC
    plt.subplot(2, 3, 5)
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, linewidth=2, label=f'AUC = {roc_auc:.4f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('Courbe ROC')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # 6. Distribution des probabilités
    plt.subplot(2, 3, 6)
    plt.hist(y_pred_proba[y_test == 0], bins=50, alpha=0.6, label='FP')
    plt.hist(y_pred_proba[y_test == 1], bins=50, alpha=0.6, label='CP')
    plt.axvline(best_threshold, color='black', linestyle='--',
                label=f'Seuil={best_threshold:.2f}')
    plt.title('Distribution des probabilités')
    plt.xlabel('Probabilité prédite')
    plt.ylabel('Fréquence')
    plt.legend()
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{output_path}{model_type}_results.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✔ Graphiques sauvegardés dans {output_path}{model_type}_results.png")

# =============================
# LANCEMENT
# =============================
train_model(model_type=model_type, data_path=data_path, target_length=target_length)